# Vulnerability and Housing Prices

In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
import re

In [27]:
"""County Demographics - taken from https://opendata.maryland.gov/Demographic/Choose-Maryland-Compare-Counties-Demographics/pa7d-u6hs"""
county_demographics = pd.read_csv("county_demographics.csv")
county_demographics.head()

,County,"Total Population, 2018","Total Population, 2010","Total Population, 2000","Population Change, 2000-2010",Population Density per Square Mile,Median Age,Per Capita Personal Income ($ Dollars),Median Household Income ($ Dollars),Total Personal Income ($ Thousands)
0,Baltimore City,602495,620961,651262,-30301,7671.5,35.6,49168,46762,30073681
1,Wicomico County,103195,98733,84638,14095,263.7,35.9,40896,52341,4209176
2,St. Mary's County,112664,105151,86196,18955,294.4,36.8,54921,82433,6187732
3,Somerset County,25675,26470,24746,1724,82.8,37.4,31327,40320,811937
4,Prince George's County,909308,863420,801091,62329,1788.8,37.5,47365,80858,43232981


In [28]:
"""Socioeconomic Characteristics per County - taken from https://opendata.maryland.gov/Demographic/Maryland-Jurisdictions-Socioeconomic-Characteristi/v67y-zu4n"""
county_socioeconomic = pd.read_csv("county_socioeconomic.csv")
county_socioeconomic.head()

,Jurisdictions,Total Households,Population 25 years and older,Less than 9th Grade,High School no Diploma,High School Diploma,Some College no degree,Associates degree,Bachelor's degree,Graduate or Professional,...,Male,Female,White Alone,Black Alone,Asian Alone,American Indian/Alaska Native Alone,Native Hawaiian/Pacific Islander Alone,Some Other Race Alone,Two or More Races,Hispanic or Latino (of any race)
0,Allegany County,28167,50793,1454,4089,21302,9999,4825,4733,4391,...,38077,34983,64695,6021,580,118,34,167,1445,1233
1,Anne Arundel County,204829,382657,8852,22100,93862,78474,28736,88157,62476,...,276842,282895,416331,89001,20103,1222,383,12470,20227,39402
2,Baltimore County,312826,569088,18308,33161,154724,111102,40101,121891,89801,...,391102,434564,519063,226879,47772,2110,394,7828,21620,40850
3,Calvert County,31479,60627,1065,3001,18704,15532,4570,10038,7717,...,44966,45561,73873,11124,1371,77,0,530,3552,3092
4,Caroline County,12010,22037,1291,2389,9207,4171,1608,2112,1259,...,15822,16831,26419,4411,187,30,16,623,967,2101


In [29]:
"""County Vulnerability - taken from https://svi.cdc.gov"""
county_vulnerability = pd.read_csv("county_vulnerability.csv")
county_vulnerability.head()

,FID,ST,STATE,ST_ABBR,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,...,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP
0,0,24,MARYLAND,MD,Anne Arundel,24003,"Anne Arundel County, Maryland",414.887094,559737.0,0.0,...,0.0,0.0,0.0,0.0,0.0,32248.0,1853.0,6.0,0.3,553075.0
1,1,24,MARYLAND,MD,Baltimore,24005,"Baltimore County, Maryland",598.361844,825666.0,0.0,...,0.0,0.0,0.0,0.0,0.0,60233.0,2259.0,7.4,0.3,821710.0
2,2,24,MARYLAND,MD,Calvert,24009,"Calvert County, Maryland",213.197148,90527.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5327.0,744.0,6.0,0.8,67719.0
3,3,24,MARYLAND,MD,Carroll,24013,"Carroll County, Maryland",447.600142,167535.0,0.0,...,0.0,0.0,0.0,0.0,0.0,7249.0,711.0,4.4,0.4,137277.0
4,4,24,MARYLAND,MD,Cecil,24015,"Cecil County, Maryland",346.276658,102175.0,0.0,...,0.0,0.0,0.0,0.0,0.0,7107.0,782.0,7.0,0.8,85648.0


In [30]:
"""New Construction - taken from https://opendata.maryland.gov/Housing/Maryland-Total-New-Parcels-For-Residential-Develop/6umw-84d2"""
county_development = pd.read_csv("county_development.csv")
county_development.head()

,Date created,Year,MARYLAND,Allegany County,Anne Arundel County,Baltimore City,Baltimore County,Calvert County,Caroline County,Carroll County,...,Kent County,Montgomery County,Prince George's County,Queen Anne's County,Somerset County,St. Mary's County,Talbot County,Washington County,Wicomico County,Worcester County
0,5/15/2017,2015,9712,16,1625,268,918,118,12,428,...,38,1186,1746,180,29,160,52,191,135,62
1,5/15/2017,2013,10346,32,1359,237,715,269,27,359,...,40,1624,1384,187,12,444,33,223,97,92
2,5/15/2017,2006,19814,137,1604,328,1813,393,321,654,...,149,1429,2920,376,167,868,287,1086,719,821
3,5/15/2017,2008,10380,112,1160,239,728,234,76,251,...,71,963,1473,205,93,624,172,356,282,328
4,5/15/2017,2012,8599,62,1171,84,648,264,25,290,...,22,1079,979,160,18,489,43,142,123,83


In [7]:
"""Find if vulnerability is correlated to less development"""

'Find if vulnerability is correlated to less development'

In [8]:
print(county_demographics.shape)
merge_name_demo = "County"

(24, 10)


In [9]:
print(county_socioeconomic.shape)
merge_name_socio = "Jurisdicions"

(24, 41)


In [19]:
print(county_vulnerability.shape)
merge_name_vuln = "COUNTY"

(24, 124)


In [20]:
"""Because data is from 2012 to 2016, the average data has been taken in this context"""

county_development = county_development.sort_values("Year")
county_2012_data = county_development.loc[(county_development["Year"] >= 2012)]
county_12_16_avg = county_2012_data.groupby("Date created").mean()
county_12_16_avg

,Year,MARYLAND,Allegany County,Anne Arundel County,Baltimore City,Baltimore County,Calvert County,Caroline County,Carroll County,Cecil County,...,Kent County,Montgomery County,Prince George's County,Queen Anne's County,Somerset County,St. Mary's County,Talbot County,Washington County,Wicomico County,Worcester County
Date created,,,,,,,,,,,,,,,,,,,,,
5/15/2017,2013.5,9710.75,31.75,1455.0,185.25,772.25,221.25,23.25,330.0,164.75,...,35.25,1328.25,1345.25,181.0,21.0,373.25,43.75,191.5,110.0,91.25


In [32]:
dc_development = pd.read_csv("dc_development.csv")
dc_residential = dc_development.loc[(dc_development["PROJECTTYPE"] == "RESIDENTIAL")]
print(dc_residential.info())

"""Count of DC residential productions is 60 in 2014. We're taking this to mean about 60 construction projects in 2014"""

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60 entries, 70 to 215
Data columns (total 13 columns):
OBJECTID            60 non-null int64
PROJECTNAME         60 non-null object
LOCATION            60 non-null object
WARD                60 non-null int64
DEVELOPER           60 non-null object
ARCHITECT           60 non-null object
STATUS              60 non-null object
UNITS               60 non-null float64
TYPE                60 non-null object
SQFT                0 non-null float64
ESTVALUEINMILION    54 non-null float64
ESTDELIVERY         60 non-null object
PROJECTTYPE         60 non-null object
dtypes: float64(3), int64(2), object(8)
memory usage: 4.7+ KB
None


"Count of DC residential productions is 60 in 2014. We're taking this to mean about 60 construction projects in 2014"

# Vulnerability Index

In [22]:
"""DC Vulnerability - taken from https://svi.cdc.gov"""
dc_vulnerability = pd.read_csv("dc_vulnerability.csv")

In [23]:
"""Merging county vulnerability and DC vulnerability to compare"""
county_combined_vulnerability = pd.concat([dc_vulnerability, county_vulnerability])

In [24]:
county_combined_vulnerability.head()

,FID,ST,STATE,ST_ABBR,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,...,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP
0,0,11,DISTRICT OF COLUMBIA,DC,District of Columbia,11001,"District of Columbia, District of Columbia",61.145040,659009.0,0.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,33419.0,1818.0,5.2,0.3,1134734.0
0,0,24,MARYLAND,MD,Anne Arundel,24003,"Anne Arundel County, Maryland",414.887094,559737.0,0.0,...,0.0,0.0,0.0,0.0,0.0,32248.0,1853.0,6.0,0.3,553075.0
1,1,24,MARYLAND,MD,Baltimore,24005,"Baltimore County, Maryland",598.361844,825666.0,0.0,...,0.0,0.0,0.0,0.0,0.0,60233.0,2259.0,7.4,0.3,821710.0
2,2,24,MARYLAND,MD,Calvert,24009,"Calvert County, Maryland",213.197148,90527.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5327.0,744.0,6.0,0.8,67719.0
3,3,24,MARYLAND,MD,Carroll,24013,"Carroll County, Maryland",447.600142,167535.0,0.0,...,0.0,0.0,0.0,0.0,0.0,7249.0,711.0,4.4,0.4,137277.0


In [25]:
mass_index = county_combined_vulnerability[["COUNTY", "E_MUNIT", "E_CROWD", "E_NOVEH", "E_GROUPQ"]]
#Combine data of parcel data

In [26]:
mass_index

,COUNTY,E_MUNIT,E_CROWD,E_NOVEH,E_GROUPQ
0,District of Columbia,140304.0,9930.0,100656.0,39967.0
0,Anne Arundel,28410.0,3264.0,8089.0,16092.0
1,Baltimore,62724.0,5144.0,24879.0,20798.0
2,Calvert,1447.0,389.0,1309.0,650.0
3,Carroll,3818.0,396.0,2687.0,3479.0
4,Cecil,2477.0,583.0,1733.0,1550.0
5,Frederick,10028.0,1354.0,4445.0,4624.0
6,Harford,10081.0,985.0,4552.0,1951.0
7,Queen Anne's,380.0,253.0,535.0,464.0
8,Washington,5428.0,1065.0,4594.0,8858.0


In [44]:
mass_index = mass_index.merge(county_demographics[["County", "Total Population, 2018"]], left_on = "COUNTY", right_on = "County", how = "outer" )

In [45]:
mass_index.head()

,COUNTY,E_MUNIT,E_CROWD,E_NOVEH,E_GROUPQ,County,"Total Population, 2018"
0,District of Columbia,140304.0,9930.0,100656.0,39967.0,NaN,NaN
1,Anne Arundel,28410.0,3264.0,8089.0,16092.0,NaN,NaN
2,Baltimore,62724.0,5144.0,24879.0,20798.0,NaN,NaN
3,Calvert,1447.0,389.0,1309.0,650.0,NaN,NaN
4,Carroll,3818.0,396.0,2687.0,3479.0,NaN,NaN


In [39]:
mass_index = county_combined_vulnerability[["COUNTY", "E_MUNIT", "E_CROWD", "E_NOVEH", "E_GROUPQ"]]
#Combine data of parcel data

In [41]:
mass_index

,COUNTY,E_MUNIT,E_CROWD,E_NOVEH,E_GROUPQ
0,District of Columbia,140304.0,9930.0,100656.0,39967.0
0,Anne Arundel,28410.0,3264.0,8089.0,16092.0
1,Baltimore,62724.0,5144.0,24879.0,20798.0
2,Calvert,1447.0,389.0,1309.0,650.0
3,Carroll,3818.0,396.0,2687.0,3479.0
4,Cecil,2477.0,583.0,1733.0,1550.0
5,Frederick,10028.0,1354.0,4445.0,4624.0
6,Harford,10081.0,985.0,4552.0,1951.0
7,Queen Anne's,380.0,253.0,535.0,464.0
8,Washington,5428.0,1065.0,4594.0,8858.0
